## Importing libraries

In [1]:
import pandas as pd
import plotly   
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State 

app = dash.Dash(__name__)


### Loading data

In [2]:
covid_Data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
covid = pd.read_csv(covid_Data_url)
covid.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,30175,30451,30616,30967,31238,31517,31836,32022,32324,32672
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2192,2269,2330,2402,2466,2535,2580,2662,2752,2819
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12445,12685,12968,13273,13571,13907,14272,14657,15070,15500
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,212,212,259,267,276,284,291,315,328,346


### Dropping fields

In [3]:
coviddrop = covid.drop(['Province/State','Lat','Long'],axis =1)
coviddrop

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,30175,30451,30616,30967,31238,31517,31836,32022,32324,32672
1,Albania,0,0,0,0,0,0,0,0,0,...,2192,2269,2330,2402,2466,2535,2580,2662,2752,2819
2,Algeria,0,0,0,0,0,0,0,0,0,...,12445,12685,12968,13273,13571,13907,14272,14657,15070,15500
3,Andorra,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,Angola,0,0,0,0,0,0,0,0,0,...,212,212,259,267,276,284,291,315,328,346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,0,0,0,0,0,0,0,0,0,...,711,712,713,713,713,714,715,717,719,719
262,Yemen,0,0,0,0,0,0,0,0,0,...,1076,1089,1103,1118,1128,1158,1190,1221,1240,1248
263,Comoros,0,0,0,0,0,0,0,0,0,...,272,272,272,272,272,303,303,303,309,309
264,Tajikistan,0,0,0,0,0,0,0,0,0,...,5691,5747,5799,5849,5900,5900,6005,6058,6058,6159


## Transforming and data Cleaning

In [4]:
Covidtransform = coviddrop.melt(id_vars = 'Country/Region')
Covidtransform.columns= ['Country','Date','No of Cases']
Covidtransform['Date'] = pd.to_datetime(Covidtransform.Date)
Covidtransform
Covidgroup = Covidtransform.groupby([pd.Grouper(key = 'Date', freq = 'M'),'Country'])['No of Cases'].sum().reset_index().sort_values(by = 'No of Cases',ascending = False)
covidGroupfilter = Covidgroup[Covidgroup['Date'] <= '2020-06-30']
covidGroupfilter

,Date,Country,No of Cases
1114,2020-06-30,US,64933225
926,2020-05-31,US,45406912
963,2020-06-30,Brazil,28161222
738,2020-04-30,US,19552396
1080,2020-06-30,Russia,16097579
...,...,...,...
219,2020-02-29,Cameroon,0
221,2020-02-29,Central African Republic,0
222,2020-02-29,Chad,0
223,2020-02-29,Chile,0


In [5]:
# Define the app
app.layout = html.Div([
     html.Div([
        dcc.Graph(id='our_graph')
    ]
         ,className='nine columns'
      
     ),
    
    html.Div([

        html.Br(),
        html.Label(['Choose no of Countries to Compare:'],style={'font-weight': 'bold', "text-align": "center",'font-size':'20px'}),
        dcc.Dropdown(id='Country_one',
            options=[{'label':x, 'value':x} for x in covidGroupfilter.sort_values('Country')['Country'].unique()],
            value=['US','Italy','Australia'],
            multi=True,
            disabled=False,
            clearable=True,
            searchable=True,
            style={"padding": 5},
            placeholder='Choose Country...',
            persistence='string',
            persistence_type='memory'),

    ]
        ,className='three columns'
    ),

])

In [6]:
@app.callback(
    Output('our_graph','figure'),
    [Input('Country_one','value')
    ]
)


def build_graph(First_Country):
    dff1 = covidGroupfilter.loc[covidGroupfilter['Country'].isin(First_Country)]
    fig = px.line(dff1, x="Date", y="No of Cases", color='Country', height=1000)
    fig.update_layout(yaxis={'title':'Known Cases'},
                      title={'text':'Biweekly COVID 19 Cases',
                      'font':{'size':28},'x':0.5,'xanchor':'center'})
    return fig



In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jul/2020 18:17:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:17:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:17:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:17:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:17:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:17:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:18:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:24:12] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1031, in dispatch
    response.set_d

127.0.0.1 - - [05/Jul/2020 18:24:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [05/Jul/2020 18:26:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 18:26:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 20:29:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 20:29:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 20:29:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2020 20:29:57] "POST /_dash-update-component HTTP/1.1" 200 -
